Problem: Region Segmentation Using Traditional Techniques  (3 Marks) \
i. Implement a region-based segmentation method (e.g., thresholding, edge
detection) to segment the mask regions for faces identified as "with mask." \
ii. Visualize and evaluate the segmentation results.

In [109]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [110]:
import cv2
import os
import json
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.cluster import KMeans
from torch.utils.data import Dataset, DataLoader, Subset
import torch

In [111]:
base_dir = "/content/drive/MyDrive/Segmentation Dataset/1"
cropped_images_dir = os.path.join(base_dir, "face_crop")
segmented_images_dir = os.path.join(base_dir, "face_crop_segmentation")
normal_images_dir = os.path.join(base_dir, "img")
json_path = os.path.join(base_dir, "mapped_images.json")

In [112]:
# Load JSON mapping
with open(json_path, 'r') as json_file:
    mapped_images = json.load(json_file)

In [113]:
def preprocess_image(image):
    if len(image.shape) == 3:  # Convert color image to grayscale if needed
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image  # Already grayscale

    gray = cv2.normalize(gray, None, 0, 255, cv2.NORM_MINMAX)  # Ensure proper range
    gray = np.uint8(gray)  # Convert to uint8
    gray = cv2.GaussianBlur(gray, (5, 5), 0)

    if gray.dtype == np.uint8:
        gray = cv2.equalizeHist(gray)  # Apply histogram equalization safely
    return gray


# ---------------------------------------
# METHOD 1: OTSU’S THRESHOLDING + EDGES


In [114]:
def segment_otsu(image):
    gray = preprocess_image(image)
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, np.ones((3, 3), np.uint8))


# METHOD 2: ADAPTIVE THRESHOLDING

In [115]:
def segment_adaptive_threshold(image):
    gray = preprocess_image(image)
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    return cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, np.ones((3, 3), np.uint8))


# METHOD 3: WATERSHED SEGMENTATION

In [116]:
def segment_watershed(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) if len(image.shape) == 3 else image
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Ensure the image is uint8 before thresholding
    blurred = cv2.normalize(blurred, None, 0, 255, cv2.NORM_MINMAX)
    blurred = np.uint8(blurred)

    _, binary = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Noise removal (Morphology)
    kernel = np.ones((3, 3), np.uint8)
    sure_bg = cv2.dilate(binary, kernel, iterations=3)

    # Foreground area
    dist_transform = cv2.distanceTransform(binary, cv2.DIST_L2, 5)

    # Normalize and convert dist_transform to uint8
    dist_transform = cv2.normalize(dist_transform, None, 0, 255, cv2.NORM_MINMAX)
    dist_transform = np.uint8(dist_transform)

    _, sure_fg = cv2.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)

    # Ensure both have the same dtype
    sure_fg = np.uint8(sure_fg)
    sure_bg = np.uint8(sure_bg)

    # Identify unknown regions
    unknown = cv2.subtract(sure_bg, sure_fg)

    # Marker labelling
    markers = cv2.connectedComponents(sure_fg)[1] + 1
    markers[unknown == 255] = 0

    # Convert markers to CV_32SC1 before passing to watershed
    markers = markers.astype(np.int32) # Ensure markers is of type int32

    # Apply Watershed
    image_color = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR) if len(image.shape) == 2 else image
    cv2.watershed(image_color, markers)

    # Return the segmented mask
    return np.uint8(markers > 1) * 255

# METHOD 4: K-MEANS CLUSTERING

In [118]:
def segment_kmeans(image, k=3):
    pixel_values = image.reshape((-1, 3)).astype(np.float32)
    _, labels, centers = cv2.kmeans(pixel_values, k, None, (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0), 10, cv2.KMEANS_RANDOM_CENTERS)
    segmented_image = centers[labels.flatten()].reshape(image.shape)
    gray = cv2.cvtColor(np.uint8(segmented_image), cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return binary

# METHOD 5: GRABCUT ALGORITHM

In [120]:
def segment_grabcut(image):
    mask = np.zeros(image.shape[:2], np.uint8)
    bgd_model = np.zeros((1, 65), np.float64)
    fgd_model = np.zeros((1, 65), np.float64)
    rect = (10, 10, image.shape[1] - 20, image.shape[0] - 20)
    cv2.grabCut(image, mask, rect, bgd_model, fgd_model, 7, cv2.GC_INIT_WITH_RECT)
    return np.where((mask == 2) | (mask == 0), 0, 255).astype('uint8')


--------------------------------------------------------------------------
# Evaluation Metrics

In [121]:
def calculate_iou(segmented_mask, ground_truth):
    intersection = np.logical_and(segmented_mask, ground_truth).sum()
    union = np.logical_or(segmented_mask, ground_truth).sum()
    return intersection / union if union else 0

def dice_coefficient(segmented_mask, ground_truth):
    intersection = np.logical_and(segmented_mask, ground_truth).sum()
    return (2 * intersection) / (segmented_mask.sum() + ground_truth.sum()) if segmented_mask.sum() + ground_truth.sum() else 0

def compute_metrics(segmented_mask, ground_truth):
    segmented_mask = segmented_mask.flatten()
    ground_truth = ground_truth.flatten()
    return precision_score(ground_truth, segmented_mask, average='macro'), recall_score(ground_truth, segmented_mask, average='macro'), f1_score(ground_truth, segmented_mask, average='macro')

    precision = precision_score(ground_truth, segmented_mask, average='macro')
    recall = recall_score(ground_truth, segmented_mask, average='macro')
    f1 = f1_score(ground_truth, segmented_mask, average='macro')

    return precision, recall, f1


# ----------------------------
# Visualization



In [122]:

def visualize_segmentation(original, segmented, ground_truth, title="Segmentation Result"):
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 3, 1)
    plt.imshow(cv2.cvtColor(original, cv2.COLOR_BGR2RGB))
    plt.title("Original Image")
    plt.axis("off")

    plt.subplot(1, 3, 2)
    plt.imshow(segmented, cmap="gray")
    plt.title("Segmented Mask")
    plt.axis("off")

    plt.subplot(1, 3, 3)
    plt.imshow(ground_truth, cmap="gray")
    plt.title("Ground Truth Mask")
    plt.axis("off")

    plt.suptitle(title)
    plt.show()

# ----------------------------
# Process Images and Evaluate


In [125]:
class MaskDataset(Dataset):
    def __init__(self, mapped_images, img_height, img_width):
        self.mapped_images = mapped_images
        self.img_height = img_height
        self.img_width = img_width

    def __len__(self):
        return len(self.mapped_images)

    def __getitem__(self, idx):
        img_path, mask_path = self.mapped_images[idx]
        # Load and resize image
        img = cv2.imread(img_path)
        if img is None:
            raise FileNotFoundError(f"⚠️ Image not found: {img_path}")
        img = cv2.resize(img, (self.img_width, self.img_height))
        img = img.astype(np.float32) / 255.0
        img = np.transpose(img, (2, 0, 1))  # (C, H, W)

        # Load and resize mask
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        if mask is None:
            raise FileNotFoundError(f"⚠️ Mask not found: {mask_path}")
        mask = cv2.resize(mask, (self.img_width, self.img_height))
        mask = np.where(mask > 0, 1, 0).astype(np.uint8)
        mask = np.expand_dims(mask, axis=0)  # (1, H, W)

        return torch.tensor(img, dtype=torch.float32), torch.tensor(mask, dtype=torch.float32)


In [126]:
import cv2
import numpy as np
import os
import json
import random
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Subset

# Set random seed for reproducibility
RANDOM_SEED = 42
random.seed(RANDOM_SEED)

# Load mapped images from JSON
with open("/content/drive/MyDrive/Segmentation Dataset/1/mapped_images.json", "r") as f:
    mapped_images = json.load(f)

# Define image dimensions and channels
IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS = 256, 256, 3
BATCH_SIZE = 32
# Initialize dataset
dataset = MaskDataset(mapped_images, IMG_HEIGHT, IMG_WIDTH)

# Get all indices
indices = list(range(len(dataset)))

# Split dataset: 70% training, 30% validation
train_indices, val_indices = train_test_split(indices, test_size=0.3, random_state=RANDOM_SEED)

# Create subsets
train_dataset = Subset(dataset, train_indices)
val_dataset = Subset(dataset, val_indices)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

print(f"✅ Data ready! Train size: {len(train_indices)}, Validation size: {len(val_indices)}")

# Define segmentation functions (Otsu, Adaptive, Watershed, GrabCut, K-Means)
# Assuming they are defined earlier in the script

def evaluate_segmentation():
    results = {"Otsu": [], "Adaptive": [], "Watershed": [], "GrabCut": [], "K-Means": []}
    count = 0

    for batch in val_loader:
        count += 1
        if count > 2:  # Limit to two batches
            break

        face_crops, ground_truth_masks = batch  # Assuming dataset returns (image, mask) pairs

        for i in range(face_crops.shape[0]):
            # Convert tensor to numpy image (assuming tensor shape is [C, H, W])
            face_crop = face_crops[i].numpy().transpose(1, 2, 0)
            face_crop = (face_crop * 255).astype(np.uint8)  # Rescale & convert to uint8 if necessary

            # Convert mask to binary format
            ground_truth_mask = ground_truth_masks[i].numpy()
            ground_truth_mask = (ground_truth_mask > 0.5).astype(np.uint8) * 255  # Ensure mask is in [0, 255]

            # Perform segmentation
            methods = {
                "Otsu": segment_otsu(face_crop),
                "Adaptive": segment_adaptive_threshold(face_crop),
                "Watershed": segment_watershed(face_crop),
                "GrabCut": segment_grabcut(face_crop),
                "K-Means": segment_kmeans(face_crop)
            }

            # Evaluate each segmentation method
            for method, seg_mask in methods.items():
                iou_score = calculate_iou(seg_mask, ground_truth_mask)
                dice_score = dice_coefficient(seg_mask, ground_truth_mask)
                precision, recall, f1 = compute_metrics(seg_mask, ground_truth_mask)

                results[method].append({
                    "IoU": iou_score, "Dice": dice_score,
                    "Precision": precision, "Recall": recall, "F1": f1
                })

    return results




✅ Data ready! Train size: 6567, Validation size: 2815


In [127]:
# Run evaluation
overall_results = evaluate_segmentation()

# Print overall metrics
for method, metrics in overall_results.items():
    avg_iou = np.mean([m["IoU"] for m in metrics])
    avg_dice = np.mean([m["Dice"] for m in metrics])
    avg_precision = np.mean([m["Precision"] for m in metrics])
    avg_recall = np.mean([m["Recall"] for m in metrics])
    avg_f1 = np.mean([m["F1"] for m in metrics])

    print(f"Method: {method}")
    print(f"Avg IoU: {avg_iou:.4f}, Avg Dice: {avg_dice:.4f}")
    print(f"Avg Precision: {avg_precision:.4f}, Avg Recall: {avg_recall:.4f}, Avg F1-Score: {avg_f1:.4f}\n")


Method: Otsu
Avg IoU: 0.2952, Avg Dice: 0.0017
Avg Precision: 0.4836, Avg Recall: 0.4880, Avg F1-Score: 0.4746

Method: Adaptive
Avg IoU: 0.3225, Avg Dice: 0.0019
Avg Precision: 0.4561, Avg Recall: 0.4645, Avg F1-Score: 0.3927

Method: Watershed
Avg IoU: 0.2300, Avg Dice: 0.0012
Avg Precision: 0.4723, Avg Recall: 0.4997, Avg F1-Score: 0.4688

Method: GrabCut
Avg IoU: 0.4823, Avg Dice: 0.0024
Avg Precision: 0.6987, Avg Recall: 0.6888, Avg F1-Score: 0.6617

Method: K-Means
Avg IoU: 0.2654, Avg Dice: 0.0015
Avg Precision: 0.4682, Avg Recall: 0.4718, Avg F1-Score: 0.4379

